In [1]:
import os, sys, time
import tifffile as tif
sys.path.append("/home/emilyjanedennis/Desktop/GitHub/rat_BrainPipe")
from tools.utils.io import makedir
from tools.registration.register import change_interpolation_order, transformix_command_line_call
from tools.registration.transform_list_of_points import modify_transform_files
from scipy.ndimage.interpolation import zoom

In [2]:
# setting paths
src = "/home/emilyjanedennis/Desktop/for_registration_to_lightsheet"
ann = os.path.join(src, "schwarz_brain_for_pra.tif")
fx = "/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/median_image.tif"

In [4]:
# need to make MRI annotation larger (~140% of atlas?) to transform to PRA
annvol = tif.imread(ann)
pra = tif.imread(fx)
zf,yf,xf = (pra.shape[0]/annvol.shape[0])*1.4, (pra.shape[1]/annvol.shape[1])*1.4, (pra.shape[2]/annvol.shape[2])*1.4
print("\nzooming...")
annvol_for_pra = zoom(annvol, (zf,yf,xf), order = 1)


zooming...


In [ ]:
# saved out annotation volume
print("\nsaving zoomed volume...")
tif.imsave(os.path.join(src, "schwarz2_for_pra_reg.tif"),
           annvol_for_pra.astype("uint16"))

In [3]:
reg = os.path.join(src, "schwarz_to_pra")
a2r = [os.path.join(reg, xx) for xx in os.listdir(reg) if "Transform" in xx]; a2r.sort()

dst = os.path.join(src, "transformed_schwarz_annotation_volume")
makedir(dst)

In [4]:
# transformix
transformfiles = modify_transform_files(transformfiles=a2r, dst=dst)
[change_interpolation_order(xx,0) for xx in transformfiles]

['/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/transformed_schwarz_annotation_volume/order0_TransformParameters.0.txt',
 '/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/transformed_schwarz_annotation_volume/order1_TransformParameters.1.txt',
 '/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/transformed_schwarz_annotation_volume/order2_TransformParameters.2.txt',
 '/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/transformed_schwarz_annotation_volume/order3_TransformParameters.3.txt']

In [5]:
# change the parameter in the transform files that outputs 16bit images instead
for fl in transformfiles:# Read in the file
    with open(fl, "r") as file:
        filedata = file.read()
    # Replace the target string
    filedata = filedata.replace('(ResultImagePixelType "float")', '(ResultImagePixelType "short")')
    # Write the file out again
    with open(fl, "w") as file:
      file.write(filedata)

In [7]:
# run transformix  
transformix_command_line_call("/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/schwarz_for_pra_reg.tif",dst, transformfiles[-1])  

Running transformix, this can take some time....

b'\ntransformix is started at Sat Aug 15 16:44:16 2020.\n\nwhich transformix:   transformix\ntransformix runs at: emilyjanedennis-System-Product-Name\n  Linux 5.4.0-26-generic (x64), #30-Ubuntu SMP Mon Apr 20 16:58:30 UTC 2020\n  with 128513 MB memory, and 1 cores @ 1200 MHz.\nRunning transformix with parameter file "/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/transformed_schwarz_annotation_volume/order3_TransformParameters.3.txt".\n\nReading the elastix parameters from file ...\n\nInstalling all components.\nInstallingComponents was successful.\n\nERROR: OpenCL-enabled device is not present!\nOpenCL processing is disabled.\nELASTIX version: 4.800\nCommand line options from ElastixBase:\n-in       /home/emilyjanedennis/Desktop/for_registration_to_lightsheet/schwarz_for_pra_reg.tif\n-out      /home/emilyjanedennis/Desktop/for_registration_to_lightsheet/transformed_schwarz_annotation_volume/\n-threads  unspecified, so all

SyntaxError: invalid syntax (<ipython-input-6-c78f95a5303f>, line 1)